In [932]:
import numpy as np
import math

    We want to implement a finite element approximation of the solution to a Dirichlet problem for the Laplace Equation. We also have implemented a mesh refinement function which splits each edge into two halves and constructs triangles and rectangles, neumann and dirichlet arrays from there. We work with a domain that is inside a square of side length =3.0 and outside of a circle of radius 1.14 centered at the bottom left corner of the square.The mesh refinement function can easily be adapted to another kind of domain.
 

In [933]:
def f (X):
    #Volume Force function
    x=-X[0]*X[1]
    return 0

In [934]:
def g (X):
    #neumann condition function
    return 0

In [935]:
def d (X):
    #dirichlet bdry condition
    y=1+X[0]*X[0]+X[1]*X[1]
    return 2.0

In [936]:
def scamult(k, mat):
    # I ran into troubles when multiplying a matrix by a scalar zero so 
    # I made my own mutliplication function
    l=len(mat)
    target=[[0.0 for i in range(l)] for i in range(l)]
    for i in range(l):
        for j in range(l):
            target[i][j]=mat[i][j]*k
    return target

In [937]:
def matadd (mat1, mat2):
    #I also ran into trouble when simply adding matrices together somehow 
    #so there we are
    l=len(mat1)
    target =[[0.0 for i in range(l)] for i in range(l)]
    for i in range(l):
        for j in range(l):
            target[i][j]=mat1[i][j]+mat2[i][j]
    return target     

In [938]:
def vectadd(u,v):
    #somehow I also run into trouble
    #when adding vectors together
    l=len(u)
    result=[0.0 for i in range(l)]
    for i in range(l):
        result[i]=u[i]+v[i]
    return result

In [939]:
def delta (j, k) : 
    #classical delta function 
    #I should have used this function more efficiently but idk
    #if I have time I'll try to make this program prettier
    x=-1
    if j==k : 
        x=1
    else:
        x=0
    return x

In [940]:
def d_Phi (ordered):
    #takes in array of nodes ordered by j
    #computes area matrix
    l=len(ordered)#print l
    if l == 3 : #triangle #print "Im a triangle in d_Phi"
        dPhi= [[ordered[1][1]-ordered[0][1], ordered[2][1]-ordered[0][1]],
               [ordered[1][2]-ordered[0][2], ordered[2][2]-ordered[0][2]]]#print dPhi
    else :#print "Im a rectangle in d_Phi"
        if l ==4 : #rectangle
            dPhi=[[ordered[1][1]-ordered[0][1], ordered[3][1]-ordered[0][1]],
                  [ordered[1][2]-ordered[0][2], ordered[3][2]-ordered[0][2]]]
        else :
            print ("error dPhi")#print dPhi
    return dPhi
            

In [941]:
def findnode (j, element) : 
    #given an integer j between 1 and 15, 
    #returns the index of the node in
    #elements [number, node 1 , node 2 , node 3]
    l = len(element)
    index=0
    for i in range (l-1): #either 0-2 or 0-3 
        k=element[i+1]
        if k ==j : 
            index=i+1
    return index

In [942]:
def node_order(j, element, coordinates):
    #takes in element array and j
    #returns array of nodes ordered wrt j
    #works for edge, triangles and rectangles
    l=len(element)
    k=findnode(j, element)#print "k"#print k
    if k ==0 :
        ordered=[[0.0 for i in range(3)] for i in range(l-1)]
    else :
        nodes=[]
        for i in range(l-1):
            nodes.append(element[i+1])#print "nodes general"#print nodes
        ordered=[]
        for i in range(l-1):#print "i"#print i
            ordered.append(coordinates[nodes[(k+i-1)%(l-1)]-1])#print ordered
    return ordered    

In [943]:
def stiffmat3(j, triangle, coordinates) : 
    #computes \nabla of delta function 
    #returns a vector
    nodes = node_order(j, triangle, coordinates)
    dPhi=d_Phi(nodes)
    A=np.linalg.det(dPhi)#print (A)
    D = [[1          ,1          ,1          ],
         [nodes[0][1],nodes[1][1],nodes[2][1]],
         [nodes[0][2],nodes[1][2],nodes[2][2]]]
    Dinv=np.linalg.inv(D)#print (Dinv)
    mat = [[0,0],
           [1,0],
           [0,1]]
    G = np.dot(Dinv, mat)#print (G)
    
    G_t=np.transpose(G)
    mat=np.dot(G, G_t)
    M=A/4*mat#print (M)    
    return M

In [944]:
def stiffmat4 (j, rectangle, coordinates) :
    
    nodes = node_order(j, rectangle, coordinates)
    dPhi=d_Phi(nodes)
    dPhi_t = np.transpose(dPhi)
    mat= dPhi_t*dPhi
    B= np.linalg.inv (mat)
    
    det=np.linalg.det(dPhi)
    
    a= [[2.0,-2.0],
        [-2.0,2.0]] 
    b=[[3.0,0.0],
       [0.0,-3.0]]    #very tidious but I couldnt find a safe 
    c=[[2.0,1.0],      #    way around
       [1.0,2.0]]
    a_=scamult(B[0][0],a)
    b_=scamult(B[0][1],b)
    c_=scamult(B[1][1],c)
    x= matadd(a_,b_)
    C=matadd(x,c_)#print C
    
    d=[[-1.0,1.0],[1.0,-1.0]]
    e=[[-1.0,-2.0],[-2.0,-1.0]]
    d_=scamult(B[0][0],d)
    e_=scamult(B[1][1],e)
    f_=scamult(-1*B[0][1],b)
    y=matadd(d_,e_)
    D=matadd(y,f_)#print D
    
    K = det/6#print K#print b*K
    
    N= [[C[0][0], C[0][1], D[0][0], D[0][1]],
        [C[1][0], C[1][1], D[1][0], D[1][1]],
        [D[0][0], D[0][1], C[0][0], C[0][1]],
        [D[1][0], D[1][1], C[1][0], C[1][1]]]#print N
    M=scamult(K, N)#print M
    return M

In [945]:
def mat_assembly (rectangles, triangles, coordinates) : #convincingly debugged
    nn=len(coordinates)
    A=[[0.0 for i in range(nn)] for i in range(nn)]#initialising   
    m=0
    n_t=len(triangles) #for each triangle
    for i in range(n_t):
        m+=1
        nodes=[triangles[i][1], triangles[i][2],triangles[i][3]] #isolate nodes number #print (nodes[0] )#print (triangles[i])
        mat=stiffmat3(nodes[0], triangles[i], coordinates) #calculate stiffness mat
        for j in range(3):
            for k in range(3):#print (nodes[j])#print (nodes[k])
                A[nodes[j]-1][nodes[k]-1]=mat[j][k] #add stiffness coeff to adequate entry in A#print (A)
    
    n_r=len(rectangles)
    for i in range(n_r):
        m+=1
        nodes = [rectangles[i][1], rectangles[i][2],rectangles[i][3],rectangles[i][4]]
        mat =stiffmat4 (nodes[0], rectangles[i], coordinates)
        for j in range(4):
            for k in range(4):
                A[nodes[j]-1][nodes[k]-1]=mat[j][k]#print (A)#print (m)
    return A

In [946]:
def gcent (nodes):
    #takesin array of coordinates of 1 element
    #returns coordinates of center of gravity of element
    l = len(nodes) #either 2, 3 or 4 #print (l)
    x=0
    y=0
    for i in range(l): #either 0-1, 0-2 or 0-3
        x+= nodes[i][1]
        y+=nodes[i][2]
    x=x/l
    y=y/l
    return [x,y]

In [947]:
def VF_el (j, element, coordinates):
    if len(element)== 4:
        k=6
    else :
        k=4
    delta=1.0
    nodes = node_order(j, element, coordinates)#print nodes
    if nodes[0][0] == 0.0:#print edge
        delta=0.0
    dPhi=d_Phi(nodes)
    det= np.linalg.det(dPhi)
    X=gcent(nodes)#print (delta, det, k, X, f(X))#print (nodes)
    Sum= delta*det/k * f(X)
    return Sum

In [948]:
def VF (j, triangles, rectangles, coordinates) :
    l_t=len(triangles)
    l_r=len(rectangles)
    Sum=0.0
    for i in range(l_t):#print "Im a triangle in VF"#print triangles[i]
        Sum+= VF_el(j, triangles[i], coordinates)#print Sum
    for i in range(l_r):#print "Im a rectangle in VF"#print rectangles[i]
        Sum+= VF_el(j, rectangles[i] , coordinates)#print Sum
    return Sum

In [949]:
def length (nodes):#print (nodes)
    l=len(nodes)
    L=0
    if l == 2:
        x=nodes[0][1]-nodes[1][1]#print (x)
        y=nodes[0][2]-nodes[1][2]#print (y)
        L=math.sqrt(x*x+y*y)#print (L)
    else :
        print ("error in norm")
    return L

In [950]:
def stress_el(j, edge, coordinates):
    k=2.0
    delta=1.0
    nodes = node_order(j, edge, coordinates)
    if nodes[0][0] == 0.0:#print edge
        delta=0.0
    L=length(nodes)
    X=gcent(nodes)
    Y=g(X)#print L/k#print Y[0]#print Y[1]
    Sum = delta* L/k * Y
    return Sum

In [951]:
def Stress (j, neumann, coordinates) :
    l_n=len(neumann)
    Sum=0.0
    for i in range(l_n):
        Sum += stress_el(j, neumann[i], coordinates)
    return Sum

In [952]:
def find_b(j, triangles, rectangles, neumann, coordinates) :
    vf=VF (j, triangles, rectangles, coordinates)
    stress = Stress (j, neumann, coordinates)
    b= vf +stress
    return b

In [953]:
def b_assembly (triangles, rectangles, neumann, coordinates):
    b=[]
    l=len(coordinates)
    for j in range(l):
        temp=find_b(j+1,triangles, rectangles, neumann, coordinates)
        b.append(temp)
    return b

In [954]:
def create_free(coordinates, dirichlet): #convincingly debbuged
    l=len(coordinates)
    l_d=len(dirichlet)
    free=[]
    for i in range(l):
        notfree=0
        for j in range(l_d):#print (j)
            test=dirichlet[j][1]
            if i+1 == test:
                notfree+=1
            test=dirichlet[j][2]
            if i+1 == test:
                notfree+=1
        if notfree==0:
            free.append(coordinates[i])
    return free

In [955]:
def reordering (free, coordinates): #I think this is ok debbuged too
    new=[]
    l=len(coordinates)
    l_f=len(free)
    for i in range(l_f):
        new.append(free[i][0])
    for i in range(l):
        test=coordinates[i][0]
        free=0
        for j in range(l_f):
            if test == new[j]:
                free+=1
        if free ==0 :
            new.append(test)
    return new

In [956]:
def find_new_a(A, new, l_f): #I have to debug this properly
    l=len(new)
    l_nf=l-l_f
    A_11=[[0.0 for i in range(l_f)] for i in range(l_f)]
    A_12=[[0.0 for i in range(l_f)] for i in range(l_nf)]
    for i in range(l_f):
        for j in range(l_f):#print (i)#print (new[i])#print(j)#print (new[j])
            A_11[i][j]=A[new[i]-1][new[j]-1]
    for i in range(l_nf):
        for j in range(l_f):
            A_12[i][j]=A[new[i+1]-1][new[j]-1]
    return [A_11, A_12]

In [957]:
def isolate_b (b, new, l_f): #this works
    new_b=[]
    for i in range(l_f):
        new_b.append(b[new[i]-1])
    return new_b

In [958]:
def find_U_D (coordinates, new, l_f):
    l_t=len(new)
    l=l_t-l_f
    U_D=[]
    for i in range(l):
        x=coordinates[new[l_f+i]-1][1]
        y=coordinates[new[l_f+i]-1][2]
        u_d=d([x,y]) #values at dirichlet nodes
        U_D.append(u_d)
    return U_D

In [959]:
def find_U (A_11, A_12, new_b, new, coordinates):
    l_f=len(new_b)
    U_D= find_U_D (coordinates, new, l_f)
    mat=np.linalg.inv(A_11)
    A_12_t=np.transpose(A_12)
    temp=np.dot(A_12_t, U_D)
    temp_2=new_b-temp
    U=np.dot(mat,temp_2)
    return U

In [960]:
def solve_free (A, b, coordinates, dirichlet) : #so I dont think this works well.
    free=create_free(coordinates, dirichlet)
    l_f=len(free)
    new=reordering (free, coordinates)
    A_list=find_new_a(A, new, l_f)#print (A, A_list[0], A_list[1])
    b_new=isolate_b (b, new, l_f)#print (b,free,b_new)
    U_free = find_U (A_list[0], A_list[1], b_new, new, coordinates)
    return U_free

In [961]:
def split_tri(triangle, coordinates): #this is GOOD
    l=len(coordinates)
    nodes=[coordinates[triangle[1]-1],coordinates[triangle[2]-1],coordinates[triangle[3]-1]]#print (nodes)
    midpoints=[]
    new=[]
    for i in range(3):#print ([nodes[(i)%3],nodes[(i+1)%3]])
        mid = gcent([nodes[(i)%3],nodes[(i+1)%3]])  
        l=len(coordinates)
        temp=[l+1, mid[0], mid[1]]
        already=0
        for j in range(l): #check this midpoint isnt already a node#print (j)
            test=coordinates[j]
            if test[1]==temp[1]:
                if test[2]==temp[2]: #if it is already a node#print ("Im already a node")
                    already=1#print (test, temp)
                    temp=test #will use it to build new triangles   
        if already ==0:
            coordinates.append(temp)
        midpoints.append(temp[0])
    for i in range(4):
        if i==3:
            new_tri=[i, midpoints[0],midpoints[1],midpoints[2]]
        else:
            new_tri=[i, nodes[i][0], midpoints[i], midpoints[(i-1)%3]]     
        new.append(new_tri)#print (coordinates)
    return [new,coordinates]

In [962]:
def split_rect (rectangle, coordinates): #this works well
    l=len(coordinates)
    nodes = [coordinates[rectangle[1]-1],coordinates[rectangle[2]-1]
             ,coordinates[rectangle[3]-1],coordinates[rectangle[4]-1]]#print (nodes)
    midpoints=[]
    new=[]
    for i in range(4):
        mid= gcent([nodes[(i)%4], nodes[(i+1)%4] ]) 
        l=len(coordinates)
        temp = [l+1, mid[0], mid[1]]
        already=0
        for j in range(l): #check this midpoint isnt already a node#print (j)
            test=coordinates[j]
            if test[1]==temp[1]:
                if test[2]==temp[2]: #if it is already a node#print ("Im already a node")
                    already=1#print (test, temp)
                    temp=test #will use it to build new rectangles
        if already ==0:
            coordinates.append(temp)            
        midpoints.append(temp[0])
    
    g=gcent(nodes)
    l_c=len(coordinates)
    coordinates.append([l_c+1,g[0],g[1]])
    midpoints.append(l_c+1)#print (midpoints)
    
    for i in range(4):#print (nodes[i][0])
        new_rect = [ i, nodes[i][0], midpoints[i], midpoints[4], midpoints[(i-1)%4]  ]
        new.append(new_rect)
    return [new,coordinates]

In [963]:
def new_bdry (coordinates, rectangles, triangles): #this is good
    l_r=len(rectangles)
    l_t=len(triangles)
    new_n=[]
    new_d=[]
    for i in range(l_t):
        for j in range(3):
            edge= [triangles[i][1+(j)%3],triangles[i][1+(j+1)%3]]
            neumann=isitneumann(edge, coordinates)
            dirichlet=isitdirichlet (edge, coordinates)
            if neumann==1:
                new_n.append([ len(new_n)+1, edge[0],edge[1]])
            if dirichlet ==2:
                new_d.append([len(new_d)+1, edge[0], edge[1]])
    for i in range(l_r):
        for j in range(4):
            edge = [rectangles[i][1+(j)%4],rectangles[i][1+(j+1)%4]]
            neumann=isitneumann(edge, coordinates)
            dirichlet=isitdirichlet (edge, coordinates)
            if neumann==1:
                new_n.append([ len(new_n)+1, edge[0],edge[1]])
            if dirichlet ==2:
                new_d.append([len(new_d)+1, edge[0], edge[1]])
    return [new_n, new_d]

In [964]:
def isitneumann (edge, coordinates): #this works
    test=0#print (edge)
    A=coordinates[edge[0]-1]#print (A)
    B=coordinates[edge[1]-1]#print (B)
    x_diff = A[1]-B[1]
    y_diff = A[2]-B[2]
    if x_diff== 0.0:
        if A[1] ==3.0 :#print ("Im on x=3")
            test=1
    if y_diff == 0.0:
        if A[2] == 0.0:#print ("Im on the x axis")
            test=1
    return test

In [965]:
def checkcircle(A,B): #debugged
    test=0
    x=[abs(A[1]-3.0), A[2]]
    y=[abs(B[1]-3.0), B[2]]
    l_x=math.sqrt (x[0]*x[0]+x[1]*x[1])
    l_y=math.sqrt (y[0]*y[0]+y[1]*y[1])
    if l_x <1.42:
        if l_y <1.42 :#print("Im in the circle")
            test=1
    return test

In [966]:
def isitdirichlet (edge, coordinates) : #this works too
    #print (edge)
    test=0
    A=coordinates[edge[0]-1]
    #print (A)
    B=coordinates[edge[1]-1]
    #print (B)
    x_diff = A[1]-B[1]
    y_diff = A[2]-B[2]
    if x_diff == 0.0 :
        if A[1] == 0.0 : #print ("Im on the y-axis")
            test=2
    if y_diff ==0.0:
        if A[2]==3.0:#print ("Im on y=3")
            test=2
    circle=checkcircle(A,B)
    if circle ==1:
        test=2
    return test

In [967]:
def mesh_refinement (coordinates, triangles, rectangles): #Im fairly convinced this works
    l_t=len(triangles)#print (l_t)
    l_r=len(rectangles)
    new_tri=[]
    new_rect=[]
    for i in range(l_t):#print (i)
        couple=split_tri(triangles[i], coordinates)
        l_n=len(couple[0])
        for j in range(l_n):
            new_tri.append(couple[0][j])
        coordinates=couple[1]
    for i in range(l_r):#print (i)
        couple=split_rect(rectangles[i], coordinates)
        l_n=len(couple[0])
        for j in range(l_n):
            new_rect.append(couple[0][j])
        coordinates=couple[1]#print (coordinates)#print (couple[0])#print (new_tri)#print (new_rect)
    new_tri=reindex(new_tri)
    new_rect=reindex(new_rect)
    #print (new_tri)
    #print (new_rect)
    #now I need to adjust the neumann and dirichlet arrays
    bdry= new_bdry (coordinates, new_rect, new_tri)
    
    return [coordinates, new_tri, new_rect, bdry[0], bdry[1]]

In [968]:
def reindex(liste):
    #takes in a list [[...],[...],[...],...,[...]]
    #indexes first subelement list[i][0] by increasing order
    l=len(liste)
    for i in range (l):
        liste[i][0]=i+1
    return liste

In [969]:
coordinates = [[1,0.0,0.0],[2,1.0,0.0],[3,1.59,0.0],[4,2.0,1.0],[5,3.0,1.41],[6,3.0,2.0],[7,3.0,3.0],
               [8,2.0,3.0],[9,1.0,3.0],[10,0.0,3.0],[11,0.0,2.0],[12,0.0,1.0],[13,1.0,1.0],[14,1.0,2.0],[15,2.0,2.0]]#print coordinates
#maybe I dont need to have 15x3 array, maybe just 15x2 because then index gives me back the first number?
triangles = [[1,2,3,13],[2,3,4,13],[3,4,5,15],[4,5,6,15]]
rectangles = [[1,1,2,13,12],[2,12,13,14,11],[3,13,4,15,14],[4,11,14,9,10],[5,14,15,8,9],[6,15,6,7,8]]
neumann= [[1,5,6],[2,6,7],[3,1,2],[4,2,3]]
dirichlet = [[1,3,4],[2,4,5],[3,7,8],[4,8,9],[5,9,10],[6,10,11],[7,11,12],[8,12,1] ]



A=mat_assembly (rectangles, triangles, coordinates)
b=b_assembly (triangles, rectangles, neumann, coordinates)
U_sol=solve_free (A, b, coordinates, dirichlet) 
print (U_sol)

new=mesh_refinement (coordinates, triangles, rectangles)

A=mat_assembly (new[2], new[1], new[0])
b=b_assembly (new[1], new[2], new[3], new[0])
U_sol=solve_free (A, b, new[0], new[4]) 
print (U_sol)





[4.10994547 3.18690199 7.35503611 4.75447458 7.6628622 ]
[-8.88178420e-16 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00
 -2.00000000e+00 -2.00000000e+00 -2.00000000e+00  0.00000000e+00
  4.44089210e-16  0.00000000e+00  5.55111512e-16  8.88178420e-16
  0.00000000e+00  0.00000000e+00  6.66133815e-16]


So I can tell there's a bug in my program because the mesh refinement should just give me move accurate values, not completely different values. I can't pin the problem though because on one hand the matrix assembly and boundary conditions assembly seem fine, and on the other the mesh refinement work well too. So maybe the problem is in the ordering of the nodes/edges after refinement, but I'm not sure how to fix it. 